In [19]:
from torchvision.datasets import Caltech256, Caltech101
import torch

caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech101_labels = Caltech101(root="datasets/caltech101", download=False).categories

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

In [20]:
# Examine the structure of the labels
print("Caltech256 labels (first 10):")
for i, label in enumerate(caltech256_labels[:10]):
    print(f"{i}: {label}")

print(f"\nTotal Caltech256 classes: {len(caltech256_labels)}")

print("\nCaltech101 labels (first 10):")
for i, label in enumerate(caltech101_labels[:10]):
    print(f"{i}: {label}")

print(f"\nTotal Caltech101 classes: {len(caltech101_labels)}")

Caltech256 labels (first 10):
0: 001.ak47
1: 002.american-flag
2: 003.backpack
3: 004.baseball-bat
4: 005.baseball-glove
5: 006.basketball-hoop
6: 007.bat
7: 008.bathtub
8: 009.bear
9: 010.beer-mug

Total Caltech256 classes: 257

Caltech101 labels (first 10):
0: Faces
1: Faces_easy
2: Leopards
3: Motorbikes
4: accordion
5: airplanes
6: anchor
7: ant
8: barrel
9: bass

Total Caltech101 classes: 101


In [21]:
# Clean up dataset labels by removing prefixes and normalizing formats
import re


def clean_caltech256_label(label):
    # Remove the number prefix (e.g., "001.", "002.")
    cleaned = re.sub(r"^\d+\.", "", label)
    # Replace hyphens with spaces
    cleaned = cleaned.replace("-", " ")
    return cleaned


def clean_caltech101_label(label):
    # Replace underscores with spaces and handle special cases
    cleaned = label.replace("_", " ")
    # Handle specific formatting issues
    cleaned = cleaned.replace(".", " ")
    return cleaned


# Clean all labels
caltech256_clean_labels = [clean_caltech256_label(label) for label in caltech256_labels]
caltech101_clean_labels = [clean_caltech101_label(label) for label in caltech101_labels]

print("Cleaned Caltech256 labels (first 10):")
for i, (original, cleaned) in enumerate(
    zip(caltech256_labels[:10], caltech256_clean_labels[:10])
):
    print(f"{i}: {original} -> {cleaned}")

print(f"\nCleaned Caltech101 labels (first 10):")
for i, (original, cleaned) in enumerate(
    zip(caltech101_labels[:10], caltech101_clean_labels[:10])
):
    print(f"{i}: {original} -> {cleaned}")

print(f"\nTotal cleaned Caltech256 classes: {len(caltech256_clean_labels)}")
print(f"Total cleaned Caltech101 classes: {len(caltech101_clean_labels)}")

Cleaned Caltech256 labels (first 10):
0: 001.ak47 -> ak47
1: 002.american-flag -> american flag
2: 003.backpack -> backpack
3: 004.baseball-bat -> baseball bat
4: 005.baseball-glove -> baseball glove
5: 006.basketball-hoop -> basketball hoop
6: 007.bat -> bat
7: 008.bathtub -> bathtub
8: 009.bear -> bear
9: 010.beer-mug -> beer mug

Cleaned Caltech101 labels (first 10):
0: Faces -> Faces
1: Faces_easy -> Faces easy
2: Leopards -> Leopards
3: Motorbikes -> Motorbikes
4: accordion -> accordion
5: airplanes -> airplanes
6: anchor -> anchor
7: ant -> ant
8: barrel -> barrel
9: bass -> bass

Total cleaned Caltech256 classes: 257
Total cleaned Caltech101 classes: 101


In [22]:
import nltk
from nltk.corpus import wordnet as wn

# Download WordNet data if not already available
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/bjoern/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
import pandas as pd
import numpy as np
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy_constructors import ManualTaxonomy, CrossPredictionsTaxonomy
from library.models import ResNetModel
from library.datasets import Caltech256MappedDataModule, Caltech101MappedDataModule
from library.taxonomy import Relationship, DomainClass

In [24]:
# Configuration Constants
CALCULATE = False  # Set to True to calculate similarity matrix from scratch
TRAIN = False  # Set to True to train models from scratch
PREDICT = False  # Set to True to generate predictions from scratch

print("Configuration:")
print(f"CALCULATE similarity matrix: {CALCULATE}")
print(f"TRAIN models: {TRAIN}")
print(f"PREDICT cross-domain: {PREDICT}")

Configuration:
CALCULATE similarity matrix: False
TRAIN models: False
PREDICT cross-domain: False


In [25]:
def calculate_wordnet_similarity(label1, label2):
    """Calculate WordNet WUP similarity between two labels.

    Uses multiple strategies to find the best synset matches:
    1. Direct label matching
    2. Individual word matching
    3. Context-based matching for compound terms
    """

    def get_synsets_for_label(label):
        """Get synsets for a label using multiple strategies."""
        synsets = []
        direct_synsets = wn.synsets(label.replace(" ", "_"))
        synsets.extend(direct_synsets)

        return synsets

    synsets1 = get_synsets_for_label(label1)
    synsets2 = get_synsets_for_label(label2)

    if not synsets1 or not synsets2:
        return 0.0

    # Calculate maximum similarity across all synset pairs
    max_similarity = 0.0
    for syn1 in synsets1:
        for syn2 in synsets2:
            try:
                sim = syn1.wup_similarity(syn2)
                if sim is not None and sim > max_similarity:
                    max_similarity = sim
            except:
                continue

    return max_similarity


# Test the function
test_similarity = calculate_wordnet_similarity("car", "automobile")
print(f"Test similarity between 'car' and 'automobile': {test_similarity}")

Test similarity between 'car' and 'automobile': 1.0


In [26]:
# Configuration for similarity matrix calculation/loading
if CALCULATE:
    print("Calculating similarity matrix between Caltech256 and Caltech101 labels...")
    print(
        f"This will compute {len(caltech256_clean_labels)} x {len(caltech101_clean_labels)} = {len(caltech256_clean_labels) * len(caltech101_clean_labels)} similarity scores"
    )

    # Create similarity matrix
    similarity_matrix = np.zeros(
        (len(caltech256_clean_labels), len(caltech101_clean_labels))
    )

    # Calculate similarities
    for i, caltech256_label in enumerate(caltech256_clean_labels):
        if i % 50 == 0:
            print(f"Processing Caltech256 class {i}/{len(caltech256_clean_labels)}")

        for j, caltech101_label in enumerate(caltech101_clean_labels):
            similarity = calculate_wordnet_similarity(
                caltech256_label, caltech101_label
            )
            similarity_matrix[i, j] = similarity

    print("Similarity matrix calculation complete!")

    # Save the similarity matrix for future use
    np.save(
        "data/wordnet_similarity_matrix_caltech256_caltech101.npy", similarity_matrix
    )
    print(
        "Similarity matrix saved to data/wordnet_similarity_matrix_caltech256_caltech101.npy"
    )

else:
    print("Loading similarity matrix from saved file...")
    try:
        similarity_matrix = np.load(
            "data/wordnet_similarity_matrix_caltech256_caltech101.npy"
        )
        print("Similarity matrix loaded successfully!")
    except FileNotFoundError:
        print("Similarity matrix file not found. Set CALCULATE=True to compute it.")
        print("Computing similarity matrix now...")

        # Fallback to calculation if file doesn't exist
        similarity_matrix = np.zeros(
            (len(caltech256_clean_labels), len(caltech101_clean_labels))
        )

        for i, caltech256_label in enumerate(caltech256_clean_labels):
            if i % 50 == 0:
                print(f"Processing Caltech256 class {i}/{len(caltech256_clean_labels)}")
            for j, caltech101_label in enumerate(caltech101_clean_labels):
                similarity = calculate_wordnet_similarity(
                    caltech256_label, caltech101_label
                )
                similarity_matrix[i, j] = similarity

        np.save(
            "data/wordnet_similarity_matrix_caltech256_caltech101.npy",
            similarity_matrix,
        )
        print("Similarity matrix calculated and saved!")

print(f"Matrix shape: {similarity_matrix.shape}")
print(f"Max similarity: {similarity_matrix.max():.4f}")
print(f"Mean similarity: {similarity_matrix.mean():.4f}")
print(
    f"Non-zero similarities: {np.count_nonzero(similarity_matrix)}/{similarity_matrix.size}"
)

Loading similarity matrix from saved file...
Similarity matrix loaded successfully!
Matrix shape: (257, 101)
Max similarity: 1.0000
Mean similarity: 0.3046
Non-zero similarities: 17622/25957


In [27]:
# Create relationships based on similarity threshold
similarity_threshold = 0.8  # Only consider high similarities to reduce noise
relationships = []

print(f"Creating relationships with similarity threshold: {similarity_threshold}")

# Domain 0: Caltech256, Domain 1: Caltech101
for caltech256_idx, caltech256_similarities in enumerate(similarity_matrix):
    for caltech101_idx, similarity in enumerate(caltech256_similarities):
        if similarity > similarity_threshold:
            # Create relationship: Caltech256 class -> Caltech101 class
            target_class = DomainClass(
                (np.intp(0), np.intp(caltech256_idx))
            )  # Caltech256
            source_class = DomainClass(
                (np.intp(1), np.intp(caltech101_idx))
            )  # Caltech101
            relationship = Relationship((target_class, source_class, float(similarity)))
            relationships.append(relationship)

print(
    f"Created {len(relationships)} relationships above threshold {similarity_threshold}"
)

# Also add reverse relationships (Caltech101 -> Caltech256)
reverse_relationships = []
for caltech101_idx in range(len(caltech101_clean_labels)):
    for caltech256_idx in range(len(caltech256_clean_labels)):
        similarity = similarity_matrix[caltech256_idx, caltech101_idx]
        if similarity > similarity_threshold:
            # Create relationship: Caltech101 class -> Caltech256 class
            target_class = DomainClass(
                (np.intp(1), np.intp(caltech101_idx))
            )  # Caltech101
            source_class = DomainClass(
                (np.intp(0), np.intp(caltech256_idx))
            )  # Caltech256
            relationship = Relationship((target_class, source_class, float(similarity)))
            reverse_relationships.append(relationship)

print(f"Created {len(reverse_relationships)} reverse relationships")

# Combine all relationships
all_relationships = relationships + reverse_relationships

# Create domain labels for visualization
domain_labels = {
    0: caltech256_clean_labels,  # Caltech256
    1: caltech101_clean_labels,  # Caltech101
}

# Create the ManualTaxonomy
manual_taxonomy = ManualTaxonomy(
    num_nodes=[len(caltech256_clean_labels), len(caltech101_clean_labels)],
    num_domains=2,
    relationships=all_relationships,
    domain_labels=domain_labels,
)

print(f"Manual taxonomy created with {len(all_relationships)} total relationships")
print(
    f"Graph has {manual_taxonomy.graph.number_of_nodes()} nodes and {manual_taxonomy.graph.number_of_edges()} edges"
)

Creating relationships with similarity threshold: 0.8
Created 234 relationships above threshold 0.8
Created 234 reverse relationships
Manual taxonomy created with 468 total relationships
Graph has 358 nodes and 468 edges


In [28]:
# Create domain mappings for model training
# Caltech256 mapping (original class index -> domain class index)
caltech256_mapping = {i: i for i in range(len(caltech256_clean_labels))}

# Caltech101 mapping (original class index -> domain class index)
caltech101_mapping = {i: i for i in range(len(caltech101_clean_labels))}

print(f"Caltech256 domain mapping: {len(caltech256_mapping)} classes")
print(f"Caltech101 domain mapping: {len(caltech101_mapping)} classes")

# Load target information for creating proper mappings
caltech256_targets = pd.read_csv("data/caltech256.csv")
caltech101_targets = pd.read_csv("data/caltech101.csv")

print(f"Caltech256 dataset size: {len(caltech256_targets)}")
print(f"Caltech101 dataset size: {len(caltech101_targets)}")

Caltech256 domain mapping: 257 classes
Caltech101 domain mapping: 101 classes
Caltech256 dataset size: 3060
Caltech101 dataset size: 867


In [29]:
def train_domain_model(dataset_class, mapping, domain_name, logger_name, model_name):
    """Train a ResNet model for a specific domain"""
    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name=logger_name)
    dataset = dataset_class(mapping=mapping)

    trainer = Trainer(
        max_epochs=50,
        logger=tb_logger if TRAIN else False,
        callbacks=[
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="val_loss",
                mode="min",
                save_top_k=1,
                filename=model_name,
                enable_version_counter=False,
            )
        ],
    )

    if TRAIN:
        model = ResNetModel(
            num_classes=len(set(mapping.values())),
            architecture="resnet50",
            optim="sgd",
            optim_kwargs={
                "lr": 0.01,
                "momentum": 0.9,
                "weight_decay": 5e-4,
            },
        )
        trainer.fit(model, datamodule=dataset)
        results = trainer.test(datamodule=dataset, ckpt_path="best")
    else:
        model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
        results = trainer.test(model, datamodule=dataset)

    print(f"{domain_name} Results: {results}")
    return results

In [30]:
# Train Caltech256 model
print("Training/Testing Caltech256 Model:")
caltech256_results = train_domain_model(
    Caltech256MappedDataModule,
    caltech256_mapping,
    "Caltech256",
    "caltech256_wordnet",
    "resnet50-caltech256-min-val-loss",
)

Training/Testing Caltech256 Model:


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/bjoern/miniconda3/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 48/48 [00:14<00:00,  3.37it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.6993464231491089
        eval_loss           1.4174646139144897
        hp_metric           0.6993464231491089
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Caltech256 Results: [{'eval_loss': 1.4174646139144897, 'eval_accuracy': 0.6993464231491089, 'hp_metric': 0.6993464231491089}]


In [31]:
# Train Caltech101 model
print("Training/Testing Caltech101 Model:")
caltech101_results = train_domain_model(
    Caltech101MappedDataModule,
    caltech101_mapping,
    "Caltech101",
    "caltech101_wordnet",
    "resnet50-caltech101-min-val-loss",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training/Testing Caltech101 Model:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 14/14 [00:03<00:00,  4.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.9158016443252563
        eval_loss           0.33412083983421326
        hp_metric           0.9158016443252563
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Caltech101 Results: [{'eval_loss': 0.33412083983421326, 'eval_accuracy': 0.9158016443252563, 'hp_metric': 0.9158016443252563}]


In [32]:
# Configuration for prediction generation - using centralized PREDICT constant

# The prediction files already exist from previous runs, so we'll use those
print("Using existing cross-domain prediction files...")

# Load prediction results from existing files
df_caltech256 = pd.read_csv(
    "data/caltech256_caltech101_caltech256_real_predictions.csv"
)
df_caltech101 = pd.read_csv(
    "data/caltech256_caltech101_caltech101_real_predictions.csv"
)

print(f"Caltech256 predictions shape: {df_caltech256.shape}")
print(f"Caltech101 predictions shape: {df_caltech101.shape}")

# Display column names to understand the structure
print(f"Caltech256 predictions columns: {df_caltech256.columns.tolist()}")
print(f"Caltech101 predictions columns: {df_caltech101.columns.tolist()}")

# Display sample data
print(f"\nCaltech256 sample data:")
print(df_caltech256.head())
print(f"\nCaltech101 sample data:")
print(df_caltech101.head())

Using existing cross-domain prediction files...
Caltech256 predictions shape: (3060, 2)
Caltech101 predictions shape: (867, 2)
Caltech256 predictions columns: ['caltech256', 'predictions_caltech101_on_caltech256']
Caltech101 predictions columns: ['caltech101', 'predictions_caltech256_on_caltech101']

Caltech256 sample data:
   caltech256  predictions_caltech101_on_caltech256
0         254                                    52
1         231                                    30
2          57                                     6
3         132                                    18
4         150                                    58

Caltech101 sample data:
   caltech101  predictions_caltech256_on_caltech101
0          81                                   178
1           8                                     9
2          53                                   178
3          92                                   229
4           0                                   252


In [33]:
# Construct taxonomy from cross-domain predictions
predicted_taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        (
            0,
            1,
            np.array(
                df_caltech101["predictions_caltech256_on_caltech101"], dtype=np.intp
            ),
        ),
        (
            1,
            0,
            np.array(
                df_caltech256["predictions_caltech101_on_caltech256"], dtype=np.intp
            ),
        ),
    ],
    domain_targets=[
        (0, np.array(df_caltech256["caltech256"], dtype=np.intp)),
        (1, np.array(df_caltech101["caltech101"], dtype=np.intp)),
    ],
    domain_labels=domain_labels,
    relationship_type="mcfp",
)

print("Taxonomy constructed from cross-domain predictions.")
print(
    f"Predicted taxonomy has {predicted_taxonomy.graph.number_of_nodes()} nodes and {predicted_taxonomy.graph.number_of_edges()} edges"
)

Taxonomy constructed from cross-domain predictions.
Predicted taxonomy has 358 nodes and 356 edges


In [34]:
# Generate and save taxonomy visualizations
print("Generating taxonomy visualizations...")

predicted_taxonomy.visualize_graph("Caltech256-Caltech101 Model Taxonomy").save_graph(
    "output/caltech256_caltech101_model_taxonomy.html"
)

manual_taxonomy.visualize_graph(
    "Caltech256-Caltech101 WordNet Ground Truth Taxonomy",
    height=2000,
    width=2000,
).save_graph("output/caltech256_caltech101_wordnet_taxonomy.html")

print("Taxonomy visualizations saved to output/ directory.")

Generating taxonomy visualizations...
Taxonomy visualizations saved to output/ directory.


In [35]:
print("Taxonomy Structure Comparison:")
print(
    f"WordNet Ground Truth - Nodes: {manual_taxonomy.graph.number_of_nodes()}, Edges: {manual_taxonomy.graph.number_of_edges()}"
)
print(
    f"Model Predictions - Nodes: {predicted_taxonomy.graph.number_of_nodes()}, Edges: {predicted_taxonomy.graph.number_of_edges()}"
)

try:
    edr = predicted_taxonomy.edge_difference_ratio(manual_taxonomy)
    precision, recall, f1 = predicted_taxonomy.precision_recall_f1(manual_taxonomy)

    print("\nCaltech256-Caltech101 WordNet Taxonomy Evaluation:")
    print(f"Edge Difference Ratio: {edr:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

except ValueError as e:
    print(f"\nEvaluation error: {e}")
    print("Still having issues with taxonomy comparison.")

Taxonomy Structure Comparison:
WordNet Ground Truth - Nodes: 358, Edges: 468
Model Predictions - Nodes: 358, Edges: 356

Caltech256-Caltech101 WordNet Taxonomy Evaluation:
Edge Difference Ratio: 0.9685
Precision: 0.1236
Recall: 0.0664
F1 Score: 0.0864


In [36]:
# Build universal taxonomies
print("Building universal taxonomies...")

predicted_taxonomy.build_universal_taxonomy()
predicted_taxonomy.visualize_graph(
    "Caltech256-Caltech101 Model Universal Taxonomy"
).save_graph("output/caltech256_caltech101_model_universal_taxonomy.html")

manual_taxonomy.build_universal_taxonomy()
manual_taxonomy.visualize_graph(
    "Caltech256-Caltech101 WordNet Universal Taxonomy"
).save_graph("output/caltech256_caltech101_wordnet_universal_taxonomy.html")

print("Universal taxonomy visualizations saved to output/ directory.")

# Summary statistics
print(f"\nFinal Summary:")
print(f"WordNet similarity threshold used: {similarity_threshold}")
print(f"Total relationships created: {len(all_relationships)}")
print(f"WordNet taxonomy edges: {manual_taxonomy.graph.number_of_edges()}")
print(f"Predicted taxonomy edges: {predicted_taxonomy.graph.number_of_edges()}")
try:
    print(f"Evaluation F1 Score: {f1:.4f}")
except:
    print("F1 Score: Not available due to evaluation issues")

Building universal taxonomies...
Universal taxonomy visualizations saved to output/ directory.

Final Summary:
WordNet similarity threshold used: 0.8
Total relationships created: 468
WordNet taxonomy edges: 663
Predicted taxonomy edges: 598
Evaluation F1 Score: 0.0864
